<a href="https://colab.research.google.com/github/charleskwakye/internshipAssignments/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Upload CSV files to make sql-lite database

In [2]:
from google.colab import files

uploaded = files.upload()


Saving customer.csv to customer.csv
Saving orders.csv to orders.csv


## Read CSV in pandas dataframe

In [5]:
import pandas as pd

customers_df = pd.read_csv("/content/customer.csv")
orders_df = pd.read_csv("/content/orders.csv")

customers_df.head(), orders_df.head()


(   customer_id           name         city
 0            1       John Doe     New York
 1            2     Jane Smith  Los Angeles
 2            3  Michael Brown      Chicago
 3            4    Emily Davis      Houston
 4            5   Chris Wilson      Phoenix,
    order_id  customer_id  order_date  amount
 0      1001            1  2023-01-15  150.50
 1      1002            2  2023-01-16   89.99
 2      1003            5  2023-01-17   45.00
 3      1004           10  2023-01-18  210.00
 4      1005            1  2023-01-20   30.25)

## Create SQL database

In [7]:
import sqlite3

conn = sqlite3.connect("shop.db")


## Load CSVs into SQL tables

In [8]:
customers_df.to_sql(
    "customers",
    conn,
    if_exists="replace",
    index=False
)

orders_df.to_sql(
    "orders",
    conn,
    if_exists="replace",
    index=False
)


100

## Verify that tables exist

In [10]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()


[('customers',), ('orders',)]

## 1. Retrieve all customers from the city "New York"

In [11]:
cursor.execute("SELECT * FROM customers WHERE city = 'New York';")
ny_customers = cursor.fetchall()
for customer in ny_customers:
    print(customer)

(1, 'John Doe', 'New York')


## 2. Find total number of orders placed.

In [12]:
cursor.execute("SELECT COUNT(*) FROM orders;")
total_orders = cursor.fetchone()[0]
print(f"Total number of orders placed: {total_orders}")

Total number of orders placed: 100


## 3. Find customers who have never placed an order (use LEFT JOIN).

In [13]:
cursor.execute("SELECT c.customer_id, c.name FROM customers c LEFT JOIN orders o ON c.customer_id = o.customer_id WHERE o.order_id IS NULL;")
customers_no_orders = cursor.fetchall()
print("Customers who have never placed an order:")
for customer in customers_no_orders:
    print(customer)

Customers who have never placed an order:
(74, 'Nicole Wood')
(75, 'Raymond Barnes')
(76, 'Catherine Ross')
(77, 'Patrick Henderson')
(78, 'Christine Coleman')
(79, 'Jack Jenkins')
(80, 'Debra Perry')
(81, 'Dennis Powell')
(82, 'Rachel Long')
(83, 'Jerry Patterson')
(84, 'Janet Hughes')
(86, 'Carolyn Washington')
(87, 'Aaron Butler')
(88, 'Maria Simmons')
(89, 'Henry Foster')
(90, 'Heather Gonzales')
(91, 'Douglas Bryant')
(92, 'Diane Alexander')
(93, 'Peter Russell')
(94, 'Julie Griffin')
(95, 'Adam Diaz')
(96, 'Joyce Hayes')
(97, 'Nathan Myers')
(98, 'Victoria Ford')


## 4. Calculate total amount spent by each customer.

In [14]:
cursor.execute("SELECT c.customer_id, c.name, SUM(o.amount) FROM customers c JOIN orders o ON c.customer_id = o.customer_id GROUP BY c.customer_id, c.name;")
amount_spent_by_customer = cursor.fetchall()
print("Total amount spent by each customer:")
for customer in amount_spent_by_customer:
    print(customer)

Total amount spent by each customer:
(1, 'John Doe', 581.0)
(2, 'Jane Smith', 339.99)
(3, 'Michael Brown', 144.5)
(4, 'Emily Davis', 155.0)
(5, 'Chris Wilson', 123.25)
(6, 'Sarah Miller', 290.0)
(7, 'David Taylor', 91.19)
(8, 'Jessica Anderson', 154.4)
(9, 'Daniel Thomas', 199.0)
(10, 'Laura Jackson', 244.5)
(11, 'James White', 572.45)
(12, 'Linda Harris', 136.95)
(13, 'Robert Martin', 225.75)
(14, 'Barbara Thompson', 399.0)
(15, 'William Garcia', 543.2)
(16, 'Elizabeth Martinez', 175.0)
(17, 'Joseph Robinson', 99.99)
(18, 'Susan Clark', 54.0)
(19, 'Matthew Rodriguez', 342.0)
(20, 'Margaret Lewis', 22.5)
(21, 'Charles Lee', 65.8)
(22, 'Dorothy Walker', 210.3)
(23, 'Christopher Hall', 45.0)
(24, 'Karen Allen', 155.25)
(25, 'Andrew Young', 418.9)
(26, 'Nancy Hernandez', 33.4)
(27, 'Joshua King', 90.0)
(28, 'Betty Wright', 21.0)
(29, 'Kevin Lopez', 440.0)
(30, 'Sandra Hill', 31.49)
(31, 'Edward Scott', 305.0)
(32, 'Ashley Green', 56.75)
(33, 'Brian Adams', 164.99)
(34, 'Kimberly Baker', 2

## 5. Retrieve top 3 highest order amounts

In [17]:
cursor.execute("SELECT amount FROM orders ORDER BY amount DESC LIMIT 3;")
highest_order_amounts = cursor.fetchall()
print("Top 3 highest order amounts:")
for amount in highest_order_amounts:
    print(f"${amount[0]:.2f}")

Top 3 highest order amounts:
$1200.00
$505.00
$500.00


## 6. Find the monthly sales trends (GROUP BY month).

In [26]:
import calendar

cursor.execute("SELECT strftime('%Y-%m', order_date) AS sales_sort_month, strftime('%Y', order_date) AS sales_year, strftime('%m', order_date) AS sales_month_num, SUM(amount) FROM orders GROUP BY sales_sort_month, sales_year, sales_month_num ORDER BY sales_sort_month;")
monthly_sales_trends = cursor.fetchall()
print("Monthly Sales Trends:")
for sort_month, year, month_num_str, total_sales in monthly_sales_trends:
    month_name = calendar.month_name[int(month_num_str)]
    display_month = f"{month_name}"
    print(f"{display_month} - ${total_sales:.2f}")

Monthly Sales Trends:
January - $1250.49
February - $2445.68
March - $2151.29
April - $2088.89
May - $2029.04
June - $1825.59
July - $2265.55
August - $133.49


## 7. Write a query to categorize orders: o amount < 500 → "Low" o 500–1000 → "Medium" o 1000 → "High"

In [27]:
cursor.execute("""SELECT
    order_id,
    amount,
    CASE
        WHEN amount < 500 THEN 'Low'
        WHEN amount >= 500 AND amount <= 1000 THEN 'Medium'
        WHEN amount > 1000 THEN 'High'
        ELSE 'Unknown'
    END AS order_category
FROM
    orders;""")
order_categories = cursor.fetchall()
print("Order Categories:")
for order in order_categories:
    print(f"Order ID: {order[0]}, Amount: ${order[1]:.2f}, Category: {order[2]}")

Order Categories:
Order ID: 1001, Amount: $150.50, Category: Low
Order ID: 1002, Amount: $89.99, Category: Low
Order ID: 1003, Amount: $45.00, Category: Low
Order ID: 1004, Amount: $210.00, Category: Low
Order ID: 1005, Amount: $30.25, Category: Low
Order ID: 1006, Amount: $125.75, Category: Low
Order ID: 1007, Amount: $99.00, Category: Low
Order ID: 1008, Amount: $500.00, Category: Medium
Order ID: 1009, Amount: $12.50, Category: Low
Order ID: 1010, Amount: $75.20, Category: Low
Order ID: 1011, Amount: $340.00, Category: Low
Order ID: 1012, Amount: $18.99, Category: Low
Order ID: 1013, Amount: $67.45, Category: Low
Order ID: 1014, Amount: $88.00, Category: Low
Order ID: 1015, Amount: $25.00, Category: Low
Order ID: 1016, Amount: $410.50, Category: Low
Order ID: 1017, Amount: $15.75, Category: Low
Order ID: 1018, Amount: $92.00, Category: Low
Order ID: 1019, Amount: $1200.00, Category: High
Order ID: 1020, Amount: $55.30, Category: Low
Order ID: 1021, Amount: $44.99, Category: Low
Orde